In [446]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from   sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

### Gradiente descendente

In [447]:
from sklearn.base import RegressorMixin
from typing       import Literal
import random      
class GradienteDescendente(RegressorMixin):
    gradient = None
    def __init__(self,solver:Literal['batch','stocastic','mini']='batch',eta=0.1,lenTheta=2,t0=5,t1=50,maxIter=1_000,n_epochs=100)->None:
        self.solver  = solver
        self.theta   = np.random.randn(lenTheta+1,1)
        self.eta     = 0.1
        self.maxIter = maxIter
        self.t0       = t0
        self.t1       = t1
        self.n_epochs = n_epochs


    def condition(self,vetor):
        if np.any(np.isnan(vetor)):    return True
        if np.any(np.isinf(vetor)):    return True
        if np.any(np.isneginf(vetor)): return True
        if np.any(np.isposinf(vetor)): return True
        return False
    
    def fit(self,X:np.array,y:np.array):
        m  = np.size(X,axis=0)
        Xb = np.c_[ np.ones((m,1)) ,X]
        
        cronogramaLearning = lambda t : self.t0  / (t+self.t1)
        match self.solver:
            case 'batch':
                for iter in range(self.maxIter):
                    gradient = Xb.T.dot(Xb.dot(self.theta) - y)
                    gradient = (2/m) * gradient 
                    
                    if self.condition(gradient) :break

                    self.theta = self.theta - self.eta * gradient 
                    
                    if self.condition(self.theta):break


            case 'stocastic':
                

                for epocas in range(self.n_epochs):
                    for iter in range(m):
                        randomIndex = np.random.randint(m)
                        xi = Xb[randomIndex:randomIndex+1,:]
                        yi = y[randomIndex:randomIndex+1,:]

                        gradient = 2 * xi.T.dot(xi.dot(self.theta) - yi)

                        if self.condition(gradient) :break

                        eta        = cronogramaLearning(epocas*m+iter)
                        self.theta = self.theta - eta*gradient  

                        if self.condition(self.theta):break

            case 'mini':
                lista  = [i for i in range(m)]
                sample  = random.sample(lista,k = int(np.ceil(m/2)))
                sample2 = list(set(lista)-set(sample))
                sample  = [sample,sample2]
                for miniLote in sample:
                    for iter in range(m):
                        xi = Xb[miniLote,:]
                        yi = y[miniLote ,:]
                        
                        gradient = 2 * xi.T.dot(xi.dot(self.theta) - yi)

                        if self.condition(gradient) :break

                        eta        = cronogramaLearning(len(miniLote)*m+iter)
                        self.theta = self.theta - eta*gradient  

                        if self.condition(self.theta):break




    
    def predict(self,X):
        soma = 0
        for i in range(np.size(X,axis=1)):
            soma += self.theta[i+1] * X[:,i] 
        return self.theta[0] + soma
    

In [448]:
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('dataset/SalárioMultipla.csv')
data = pd.get_dummies(data,columns=['Nivel_Educacao'],dtype=int)

data[['Experiencia','Idade']] = StandardScaler().fit_transform(data[['Experiencia','Idade']])

In [449]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import PolynomialFeatures
X = data.drop(columns='Salario')
Y = data['Salario'].values.reshape(-1,1)
xtrain, xtest, ytrain, ytest = train_test_split(X.values,Y,test_size=6,random_state=42)

In [450]:
grad = GradienteDescendente('mini',lenTheta=np.size(xtrain,axis=1))
grad.fit(xtrain,ytrain)

In [451]:
pred = grad.predict(xtest)

In [452]:
r2_score(ytest,pred),mean_absolute_error(ytest,pred)

(0.9731724642818802, 255.1863262974331)

In [453]:
pd.DataFrame({'PRED':pred.tolist(),'TRUE':ytest.tolist()})

,PRED,TRUE
0,5405.118521,[5100]
1,7939.177218,[7900]
2,10797.541857,[10400]
3,4815.375635,[4900]
4,6150.559082,[6300]
5,8655.215078,[8100]
